## Building LSST Tilesets using Datashader

Datashader provides `render_tiles` which is a utility function for creating tilesets from arbitrary datashader pipelines.

In [ ]:
import pandas as pd
import datashader as ds
import numpy as np

from datashader.tiles import render_tiles

In [ ]:
import geopandas as gpd

In [ ]:
!conda install -y -c conda-forge descartes

### Unreleased Datashader Code (remove after next datashader release)

In [ ]:
world = gpd.read_file(
    gpd.datasets.get_path('naturalearth_lowres')
)
world = world.to_crs("EPSG:3857") 
world.plot(figsize=(15,15))

In [ ]:
#from datashader.tiles import tile_previewer
#the import above is not yet released in datashader, so I'm including it here...
def tile_previewer(full_extent, tileset_url,
                   output_dir=None,
                   filename='index.html',
                   title='Datashader Tileset',
                   min_zoom=0, max_zoom=40,
                   height=None, width=None,
                   **kwargs):
    '''Helper function for creating a simple Bokeh figure with
    a WMTS Tile Source.

    Notes
    -----
    - if you don't supply height / width, stretch_both sizing_mode is used.
    - supply an output_dir to write figure to disk.
    '''

    try:
        from bokeh.plotting import figure
        from bokeh.models.tiles import WMTSTileSource
        from bokeh.io import output_file, save
        from os import path
    except ImportError:
        raise ImportError('conda install bokeh to enable creation of simple tile viewer')

    if output_dir:
        output_file(filename=path.join(output_dir, filename),
                    title=title)

    xmin, ymin, xmax, ymax = full_extent

    if height and width:
        p = figure(width=width, height=height,
                   x_range=(xmin, xmax),
                   y_range=(ymin, ymax),
                   tools="pan,wheel_zoom,reset", **kwargs)
    else:
        p = figure(sizing_mode='stretch_both',
                   x_range=(xmin, xmax),
                   y_range=(ymin, ymax),
                   tools="pan,wheel_zoom,reset", **kwargs)

    p.background_fill_color = 'black'
    p.grid.grid_line_alpha = 0
    p.axis.visible = True

    tile_source = WMTSTileSource(url=tileset_url,
                                 min_zoom=min_zoom,
                                 max_zoom=max_zoom)
    p.add_tile(tile_source, render_parents=False)

    if output_dir:
        save(p)

    return p

In [ ]:
world.geometry.bounds

### Initial data preprocessing

In [ ]:
from spatialpandas import GeoDataFrame
import numpy as np

# TODO: add fields necessary for psfmag calc.
df = world
bounds = world.geometry.bounds

full_extent_of_data = (bounds.minx.min(),
                       bounds.miny.min(),
                       np.nanmax(bounds.maxx[np.isfinite(bounds.maxx)]),
                       np.nanmax(bounds.maxy[np.isfinite(bounds.maxy)]))
full_extent_of_data

In [ ]:
x_range = (-10e6, 10e6)
y_range = (-10e6, 10e6)

# Thuy: start here and intersect world polygon to mimic load_data_func
bounding_box = (x_range[0], y_range[0], x_range[1], y_range[1])
                                                
bbox_df = gpd.GeoDataFrame(gpd.GeoSeries(bounding_box), columns=['geometry'])
#intersection_df = gpd.overlay(bbox_df, world, how='intersection')
#intersection_df.plot()
bbox_df

## Define Tiling Component Functions

### Create `load_data_func`
- accepts `x_range` and `y_range` arguments which correspond to the ranges of the supertile being rendered.
- returns a dataframe-like object (pd.Dataframe / dask.Dataframe)
- this example `load_data_func` creates a pandas dataframe with `x` and `y` fields sampled from a wald distribution 

In [ ]:
import pandas as pd
import numpy as np

def load_data_func(x_range, y_range):
    bounding_box = (x_range[0], y_range[0], x_range[1, y_range[1]])
    bbox_df = gpd.GeoDataFrame(gpd.GeoSeries(bounding_box), columns=['geometry'])
    intersection_df = gpd.overlay(world, bbox_df, how='intersection')

### Create `rasterize_func`
- accepts `df`, `x_range`, `y_range`, `height`, `width` arguments which correspond to the data, ranges, and plot dimensions of the supertile being rendered.
- returns an `xr.DataArray` object representing the aggregate.

In [ ]:
import datashader as ds

def rasterize_func(df, x_range, y_range, height, width):
    # aggregate
    cvs = ds.Canvas(x_range=x_range, y_range=y_range,
                    plot_height=height, plot_width=width)
    agg = cvs.points(df, 'x', 'y')
    return agg

### Create `shader_func`
- accepts `agg (xr.DataArray)`, `span (tuple(min, max))`.  The span argument can be used to control color mapping / auto-ranging across supertiles.
- returns an `ds.Image` object representing the shaded image.

In [ ]:
import datashader.transfer_functions as tf
from datashader.colors import viridis

def shader_func(agg, span=None):
    img = tf.shade(agg, cmap=['black','white'], span=span, how='log')
    img = tf.set_background(img, 'black')
    return img

### Create `post_render_func`
- accepts `img `, `extras` arguments which correspond to the output PIL.Image before it is write to disk (or S3), and addtional image properties.
- returns image `(PIL.Image)`
- this is a good place to run any non-datashader-specific logic on each output tile.

In [ ]:
def post_render_func(img, **kwargs):
    info = "x={},y={},z={}".format(kwargs['x'], kwargs['y'], kwargs['z'])
    return img

## Render tiles to local filesystem

In [ ]:
full_extent_of_data = (np.nanmin(df.x), np.nanmin(df.y),
                       np.nanmax(df.x), np.nanmax(df.y))

output_path = '/Users/bcollins/temp/9615_forced.tiles/'
results = render_tiles(full_extent_of_data,
                       range(25, 39),
                       load_data_func=load_data_func,
                       rasterize_func=rasterize_func,
                       shader_func=shader_func,
                       post_render_func=post_render_func,
                       output_path=output_path)

### Preview the tileset using Bokeh
- Browse to the tile output directory and start an http server:

```bash
$> cd test_tiles_output
$> python -m http.server

Starting up http-server, serving ./
Available on:
  http://127.0.0.1:8080
  http://192.168.1.7:8080
Hit CTRL-C to stop the server
```

- build a `bokeh.plotting.Figure`

In [ ]:
from bokeh.io import output_notebook, show
output_notebook()

tileset_url = 'https://lsst-tilesets.s3.amazonaws.com/9615-forced-test/{Z}/{X}/{Y}.png'

fig = tile_previewer(full_extent=full_extent_of_data,
                     height=800, width=800,
                     tileset_url=tileset_url)
show(fig)

## Render tiles to Amazon Simple Storage Service (S3)

To render tiles directly to S3, you only need to use the `s3://` protocol in your `output_path` argument

- Requires AWS Access / Secret Keys with appropriate IAM permissions for uploading to S3.
- Requires extra `boto3` dependency:
```bash
conda install boto3
```

### Configuring credentials

- Quoting [`boto3 documentation regarding credential handling`](https://boto3.readthedocs.io/en/latest/guide/configuration.html):

> The mechanism in which boto3 looks for credentials is to search through a list of possible locations and stop as soon as it finds credentials. The order in which Boto3 searches for credentials is:
1. ~~Passing credentials as parameters in the boto.client() method~~
- ~~Passing credentials as parameters when creating a Session object~~
- **Environment variables**
- **Shared credential file (~/.aws/credentials)**
- **AWS config file (~/.aws/config)**
- **Assume Role provider**
- **Boto2 config file (/etc/boto.cfg and ~/.boto)**
- **Instance metadata service on an Amazon EC2 instance that has an IAM role configured**.

- Datashader's `render_tiles` function supports only credential search locations highlighted in bold above
- **NOTE**:  all tiles written to S3 are marked with `public-read` ACL settings.

#### Setup tile bucket using AWS CLI

```bash
$> aws s3 mb s3://datashader-tiles-testing/
```

In [ ]:
output_path = 's3://lsst-tilesets/9615-forced-test/'
results = render_tiles(full_extent_of_data,
                       range(25, 39),
                       load_data_func=load_data_func,
                       rasterize_func=rasterize_func,
                       shader_func=shader_func,
                       post_render_func=post_render_func,
                       output_path=output_path)

### Preview S3 Tiles

In [ ]:
xmin, ymin, xmax, ymax = full_extent_of_data

p = figure(width=800, height=800, 
           x_range=(int(-20e6), int(20e6)),
           y_range=(int(-20e6), int(20e6)),
           tools="pan,wheel_zoom,reset")
p.axis.visible = False
p.background_fill_color = 'black'
p.grid.grid_line_alpha = 0
p.add_tile(WMTSTileSource(url="https://datashader-tiles-testing.s3.amazonaws.com/wald_tiles/{Z}/{X}/{Y}.png"),
           render_parents=False)
show(p)